In [39]:
import xgboost as xgb
import pandas as pd
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [33]:
def read_dataframe(path):
  df = pd.read_parquet(path)
  df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
  df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.seconds / 60

  df = df[((df.duration >=1) & (df.duration <=60))]
  return df

In [34]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

valid_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(valid_dicts)

target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

In [35]:
# Ref: https://medium.com/analytics-vidhya/hyperparameter-tuning-hyperopt-bayesian-optimization-for-xgboost-and-neural-network-8aedf278a1c9

Hyperopt functions:

- hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
- hp.randint(label, upper) — Returns a random integer between the range [0, upper).
- hp.uniform(label, low, high) — Returns a value uniformly between low and high.
- hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
- hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [49]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

Step 1: Initialize space or a required range of values:

In [56]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

Step 2: Define objective function:

In [57]:
# Regression: 
def objective(params):
    booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print ("SCORE:", rmse)
    #change the metric if you like
    return {'loss': rmse, 'status': STATUS_OK}

Step 3: Run Hyperopt function:

In [59]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

print(best_result)

[0]	validation-rmse:19.85710                                                                                           

[1]	validation-rmse:18.62955                                                                                           

[2]	validation-rmse:17.50175                                                                                           

[3]	validation-rmse:16.46704                                                                                           

[4]	validation-rmse:15.52083                                                                                           

[5]	validation-rmse:14.65368                                                                                           

[6]	validation-rmse:13.86190                                                                                           

[7]	validation-rmse:13.13756                                                                                           

[8]	validation-rmse:12.47774    

[67]	validation-rmse:6.23482                                                                                           

[68]	validation-rmse:6.23243                                                                                           

[69]	validation-rmse:6.22947                                                                                           

[70]	validation-rmse:6.22714                                                                                           

[71]	validation-rmse:6.22493                                                                                           

[72]	validation-rmse:6.22270                                                                                           

[73]	validation-rmse:6.22100                                                                                           

[74]	validation-rmse:6.21914                                                                                           

[75]	validation-rmse:6.21737    

[32]	validation-rmse:6.62898                                                                                           

[33]	validation-rmse:6.62971                                                                                           

[34]	validation-rmse:6.62985                                                                                           

[35]	validation-rmse:6.63030                                                                                           

[36]	validation-rmse:6.63127                                                                                           

[37]	validation-rmse:6.63152                                                                                           

[38]	validation-rmse:6.63328                                                                                           

[39]	validation-rmse:6.63488                                                                                           

[40]	validation-rmse:6.63535    

[16]	validation-rmse:6.36463                                                                                           

[17]	validation-rmse:6.35399                                                                                           

[18]	validation-rmse:6.34760                                                                                           

[19]	validation-rmse:6.33792                                                                                           

[20]	validation-rmse:6.33505                                                                                           

[21]	validation-rmse:6.32843                                                                                           

[22]	validation-rmse:6.32329                                                                                           

[23]	validation-rmse:6.31550                                                                                           

[24]	validation-rmse:6.31410    

[83]	validation-rmse:6.23114                                                                                           

[84]	validation-rmse:6.23095                                                                                           

[85]	validation-rmse:6.23108                                                                                           

[86]	validation-rmse:6.23160                                                                                           

[87]	validation-rmse:6.23203                                                                                           

[88]	validation-rmse:6.23349                                                                                           

[89]	validation-rmse:6.23370                                                                                           

[90]	validation-rmse:6.23215                                                                                           

[91]	validation-rmse:6.23238    

...

[85]	validation-rmse:6.19331                                                                                           

[86]	validation-rmse:6.19425                                                                                           

[87]	validation-rmse:6.19455                                                                                           

[88]	validation-rmse:6.19412                                                                                           

[89]	validation-rmse:6.19452                                                                                           

[90]	validation-rmse:6.19498                                                                                           

[91]	validation-rmse:6.19538                                                                                           

[92]	validation-rmse:6.19553                                                                                           

[93]	validation-rmse:6.19566    

[50]	validation-rmse:6.35003                                                                                           

[51]	validation-rmse:6.35026                                                                                           

[52]	validation-rmse:6.35136                                                                                           

[53]	validation-rmse:6.35241                                                                                           

[54]	validation-rmse:6.35203                                                                                           

[55]	validation-rmse:6.35363                                                                                           

[56]	validation-rmse:6.35481                                                                                           

[57]	validation-rmse:6.35571                                                                                           

[58]	validation-rmse:6.35705    

[26]	validation-rmse:6.84086                                                                                           

[27]	validation-rmse:6.78030                                                                                           

[28]	validation-rmse:6.72659                                                                                           

[29]	validation-rmse:6.67946                                                                                           

[30]	validation-rmse:6.63828                                                                                           

[31]	validation-rmse:6.60351                                                                                           

[32]	validation-rmse:6.57104                                                                                           

[33]	validation-rmse:6.54274                                                                                           

[34]	validation-rmse:6.51900    

[93]	validation-rmse:6.32040                                                                                           

[94]	validation-rmse:6.32130                                                                                           

[95]	validation-rmse:6.32183                                                                                           

[96]	validation-rmse:6.32261                                                                                           

[97]	validation-rmse:6.32273                                                                                           

[98]	validation-rmse:6.32326                                                                                           

[99]	validation-rmse:6.32412                                                                                           

SCORE:                                                                                                                 
6.3241211901982295               

[58]	validation-rmse:6.42320                                                                                           

[59]	validation-rmse:6.41123                                                                                           

[60]	validation-rmse:6.40084                                                                                           

[61]	validation-rmse:6.39055                                                                                           

[62]	validation-rmse:6.38089                                                                                           

[63]	validation-rmse:6.37274                                                                                           

[64]	validation-rmse:6.36412                                                                                           

[65]	validation-rmse:6.35581                                                                                           

[66]	validation-rmse:6.34815    

*Re-train the model algorithm using the best parameters obtained using hyperopt and evaluate it against the test-set or use it for the prediction*